In [1]:
import qcodes as qc
station = qc.Station(config_file='config/station.yaml')

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : /Users/caijiaqi/.qcodes/logs/command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : /Users/caijiaqi/.qcodes/logs/251105-7714-qcodes.log


In [2]:
dac1 = station.load_mock_dac1()
dac2 = station.load_mock_dac2()

In [3]:
%load_ext instrmcp.extensions

2025-11-05 14:14:31,300 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 110 ¦ /Users/caijiaqi/GitHub/instrMCP/instrmcp/servers/qcodes/station_init.py:10: QCoDeSDeprecationWarning: The `qcodes.instrument.base` module is deprecated. Please consult the api documentation at https://microsoft.github.io/Qcodes/api/index.html for alternatives.
  from qcodes.instrument.base import Instrument

2025-11-05 14:14:31,595 ¦ qcodes.station ¦ WARNING ¦ station ¦ load_config_file ¦ 414 ¦ Could not load default config for Station: 
File /Users/caijiaqi/GitHub/MeasureIt\cfg\default.station.yaml not found. 
You can change the default config file in `qcodesrc.json`.
2025-11-05 14:14:31,614 ¦ qcodes.station ¦ WARNING ¦ station ¦ load_config_file ¦ 414 ¦ Could not load default config for Station: 
File /Users/caijiaqi/GitHub/MeasureIt\cfg\default.station.yaml not found. 
You can change the default config file in `qcodesrc.json`.
2025-11-05 14:14:31,614 ¦ qcodes.station ¦ WARNING ¦ station ¦ load_config_f


MeasureIt data directory: /Users/caijiaqi/GitHub/MeasureIt
To change: measureit.set_data_dir('/path')

✅ QCoDeS MCP extension loaded
🛡️  Default mode: safe
📋 Use %mcp_status to check server status
⚠️  Use %mcp_unsafe to switch to unsafe mode (if needed)
🚀 Use %mcp_start to start the server


In [4]:
%mcp_status

🛡️ MCP Server Status:
   Desired Mode: safe
   Server Instance: ❌ Not created yet
   Available: %mcp_start


In [5]:
%mcp_option add measureit database auto_correct_json

✅ Added: measureit
✅ Added: database
✅ Added: auto_correct_json
✅ Changes will take effect when server starts


In [6]:
%mcp_unsafe

⚠️  Mode set to unsafe
⚠️  UNSAFE MODE: execute_editing_cell tool will be available
✅ Mode will take effect when server starts


In [7]:
%mcp_start

🚀 Starting MCP server...
🚀 QCoDeS MCP Server running on http://127.0.0.1:8123
🔑 Access token: 4zmj7XinLzt6D-EcK-kjhC6sgvMxG0PB25Sr-op0jo4


INFO:     Started server process [7714]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8123 (Press CTRL+C to quit)


✅ MCP server started in ⚠️ unsafe mode
⚠️  UNSAFE MODE: execute_editing_cell tool is available
INFO:     127.0.0.1:51852 - "DELETE /mcp HTTP/1.1" 400 Bad Request
INFO:     127.0.0.1:51852 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51852 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:51852 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51885 - "DELETE /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51852 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51852 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:51852 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51885 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51885 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51885 - "POST /mcp HTTP/1.1" 200 OK


In [9]:
# Sweep1D - Sweep mock_dac1 from 0 to 1 with step 0.01
import os
from MeasureIt.sweep1d import Sweep1D
from MeasureIt.util import init_database

# Configure sweep
s = Sweep1D(
    dac1.ch01.voltage,      # Parameter to sweep (mock_dac1 channel 1 voltage)
    start=0.0,              # Start value
    stop=1.0,               # Stop value
    rate=0.1,               # Sweep rate: 0.1 V/s (gives step of 0.01 with inter_delay=0.1)
    inter_delay=0.1,        # Delay between measurements (0.1s gives step of 0.01)
    save_data=True,
    bidirectional=False,    # One direction sweep
    continual=False         # Stop after one sweep
)

# Set parameters to follow (add measurements you want to record)
s.follow_param(
    dac1.ch02.voltage,      # Example: follow channel 2 voltage
    # Add more parameters as needed
)

# Initialize database
database_name = "measurements.db"
exp_name = "dac_sweep"
sample_name = "mock_dac1_sweep"
init_database(database_name, exp_name, sample_name, s)

# Start sweep
%gui qt
s.start()

# Control: Press ESC to stop, spacebar to reverse direction

2

In [13]:
# Check VISA resources
try:
    import pyvisa
    rm = pyvisa.ResourceManager()
    resources = rm.list_resources()
    print("Available VISA Resources:")
    for res in resources:
        print(f"  - {res}")
except Exception as e:
    print(f"Error checking VISA resources: {e}")

# Check QCoDeS instruments
try:
    import qcodes as qc
    instruments = qc.Instrument._all_instruments
    print("\nAvailable QCoDeS Instruments:")
    if instruments:
        for name, inst in instruments.items():
            print(f"  - {name}: {type(inst).__name__}")
    else:
        print("  No instruments currently instantiated")
except Exception as e:
    print(f"Error checking QCoDeS instruments: {e}")

Error checking VISA resources: Could not locate a VISA implementation. Install either the IVI binary or pyvisa-py.

Available QCoDeS Instruments:
  - mock_dac1: MockDAC
  - mock_dac2: MockDAC


In [11]:
# Check VISA resources
try:
    import pyvisa
    rm = pyvisa.ResourceManager()
    resources = rm.list_resources()
    print("Available VISA Resources:")
    for res in resources:
        print(f"  - {res}")
except Exception as e:
    print(f"Error checking VISA resources: {e}")

# Check QCoDeS instruments
try:
    import qcodes as qc
    instruments = qc.Instrument._all_instruments
    print("\nAvailable QCoDeS Instruments:")
    if instruments:
        for name, inst in instruments.items():
            print(f"  - {name}: {type(inst).__name__}")
    else:
        print("  No instruments currently instantiated")
except Exception as e:
    print(f"Error checking QCoDeS instruments: {e}")

Error checking VISA resources: Could not locate a VISA implementation. Install either the IVI binary or pyvisa-py.

Available QCoDeS Instruments:
  - mock_dac1: MockDAC
  - mock_dac2: MockDAC


In [ ]:
111